In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


In [2]:
# Load the trained model
model = load_model('model.h5')

# Load the label encoder
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

#load the one hot encoder
with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

# Load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
# Load the sample test data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
# Encoding 'Geography'Geography 
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded

c:\Users\a7207\OneDrive - Axtria\Documents Back Up\Upskilling\Udemy_KN_GenAI\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
# Combine the encoded 'Geography' with the rest of the input data
input_data_df = pd.concat([pd.DataFrame([input_data]), geo_encoded], axis=1)
input_data_df = input_data_df.drop(columns=['Geography'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
# Encode 'Gender' column
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
# Scale the input data
input_data_scaled = scaler.transform(input_data_df) 
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [11]:
# Make prediction
prediction = model.predict(input_data_scaled)
prediction_probability = prediction[0][0]
prediction_probability




1/1 [==============================] - 0s 45ms/step


0.05362162

In [13]:
if prediction_probability > 0.5:
    prediction_class = 'Churn'
else:
    prediction_class = 'Not Churn'
print(f"Prediction Probability: {prediction_probability:.4f}")
print(f"Prediction Class: {prediction_class}")

Prediction Probability: 0.0536
Prediction Class: Not Churn
